In [3]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("df-ops").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/09 09:13:51 INFO SparkEnv: Registering MapOutputTracker
23/09/09 09:13:51 INFO SparkEnv: Registering BlockManagerMaster
23/09/09 09:13:51 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/09/09 09:13:52 INFO SparkEnv: Registering OutputCommitCoordinator


In [5]:
df=spark.read.csv("gs://spark-fs/stackoverflowposts.csv",header=True)

In [6]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- body: string (nullable = true)
 |-- accepted_answer_id: string (nullable = true)
 |-- answer_count: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- community_owned_date: string (nullable = true)
 |-- creation_date: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- last_activity_date: string (nullable = true)
 |-- last_edit_date: string (nullable = true)
 |-- last_editor_display_name: string (nullable = true)
 |-- last_editor_user_id: string (nullable = true)
 |-- owner_display_name: string (nullable = true)
 |-- owner_user_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- post_type_id: string (nullable = true)
 |-- score: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- view_count: string (nullable = true)



In [10]:
df.show(5)
#for full data use - data.show(5,truncate=False)

+--------+--------------------+--------------------+------------------+------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+------------------------+-------------------+------------------+-------------+---------+------------+-----+--------------------+----------+
|      id|               title|                body|accepted_answer_id|answer_count|comment_count|community_owned_date|       creation_date|favorite_count|  last_activity_date|      last_edit_date|last_editor_display_name|last_editor_user_id|owner_display_name|owner_user_id|parent_id|post_type_id|score|                tags|view_count|
+--------+--------------------+--------------------+------------------+------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+------------------------+-------------------+------------------+-------------+---------+------------+-----+--------------------+-

In [12]:
#for viewing selected columns
df.select(['id','tags','score','view_count','creation_date']).show(6,False)

+--------+---------------------------------------------+-----+----------+------------------------------+
|id      |tags                                         |score|view_count|creation_date                 |
+--------+---------------------------------------------+-----+----------+------------------------------+
|452884  |python|parsing|beautifulsoup                 |1    |1178      |2009-01-17 06:10:41.080000 UTC|
|92290   |visual-studio                                |1    |225       |2008-09-18 12:57:48.340000 UTC|
|6008898 |objective-c|c|printf|nslog                   |0    |217       |2011-05-15 14:11:23.250000 UTC|
|36737743|gradle                                       |1    |9         |2016-04-20 08:15:03.563000 UTC|
|3566966 |wpf|mvvm|datagrid|styles|observablecollection|1    |100       |2010-08-25 14:34:26.310000 UTC|
|20683007|java|selenium-webdriver                      |3    |2502      |2013-12-19 13:27:11.620000 UTC|
+--------+---------------------------------------------

In [13]:
df_sub = df.select(['id','tags','score','view_count','creation_date'])

In [17]:
df_sub.filter(
    (df.score==1) & (df.view_count>500)
).show(7,False)

+--------+------------------------------------------------------------------+-----+----------+------------------------------+
|id      |tags                                                              |score|view_count|creation_date                 |
+--------+------------------------------------------------------------------+-----+----------+------------------------------+
|452884  |python|parsing|beautifulsoup                                      |1    |1178      |2009-01-17 06:10:41.080000 UTC|
|31008726|symfony2|doctrine|many-to-many|query-builder                      |1    |1268      |2015-06-23 16:46:37.273000 UTC|
|9223133 |ios4                                                              |1    |546       |2012-02-10 05:25:57.230000 UTC|
|11711033|ruby-on-rails                                                     |1    |554       |2012-07-29 16:47:16.277000 UTC|
|15565909|matlab|user-interface                                             |1    |708       |2013-03-22 08:34:46.9230

In [20]:
df_sub.where((df_sub.score==1) & (df.view_count>500)).show(5,False)

+--------+--------------------------------------------+-----+----------+------------------------------+
|id      |tags                                        |score|view_count|creation_date                 |
+--------+--------------------------------------------+-----+----------+------------------------------+
|452884  |python|parsing|beautifulsoup                |1    |1178      |2009-01-17 06:10:41.080000 UTC|
|31008726|symfony2|doctrine|many-to-many|query-builder|1    |1268      |2015-06-23 16:46:37.273000 UTC|
|9223133 |ios4                                        |1    |546       |2012-02-10 05:25:57.230000 UTC|
|11711033|ruby-on-rails                               |1    |554       |2012-07-29 16:47:16.277000 UTC|
|15565909|matlab|user-interface                       |1    |708       |2013-03-22 08:34:46.923000 UTC|
+--------+--------------------------------------------+-----+----------+------------------------------+
only showing top 5 rows



In [25]:
#Type Casting
from pyspark.sql.types import IntegerType
df_sub.withColumn("id_new",df_sub["id"].cast(IntegerType())).show(5,False)

+--------+---------------------------------------------+-----+----------+------------------------------+--------+
|id      |tags                                         |score|view_count|creation_date                 |id_new  |
+--------+---------------------------------------------+-----+----------+------------------------------+--------+
|452884  |python|parsing|beautifulsoup                 |1    |1178      |2009-01-17 06:10:41.080000 UTC|452884  |
|92290   |visual-studio                                |1    |225       |2008-09-18 12:57:48.340000 UTC|92290   |
|6008898 |objective-c|c|printf|nslog                   |0    |217       |2011-05-15 14:11:23.250000 UTC|6008898 |
|36737743|gradle                                       |1    |9         |2016-04-20 08:15:03.563000 UTC|36737743|
|3566966 |wpf|mvvm|datagrid|styles|observablecollection|1    |100       |2010-08-25 14:34:26.310000 UTC|3566966 |
+--------+---------------------------------------------+-----+----------+---------------

In [27]:
df_sub.withColumn("id_new",df_sub["id"].cast(IntegerType())).printSchema()

root
 |-- id: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- score: string (nullable = true)
 |-- view_count: string (nullable = true)
 |-- creation_date: string (nullable = true)
 |-- id_new: integer (nullable = true)



In [30]:
from pyspark.sql.types import TimestampType
df_sub.withColumn("creation_time_new",df_sub["creation_date"].cast(TimestampType())).printSchema()

root
 |-- id: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- score: string (nullable = true)
 |-- view_count: string (nullable = true)
 |-- creation_date: string (nullable = true)
 |-- creation_time_new: timestamp (nullable = true)



In [ ]:
# '\' is the new line character

In [34]:
from pyspark.sql.functions import year,month,dayofmonth

df_sub \
.withColumn("creation_year",year(df_sub["creation_date"])) \
.withColumn("creation_month",month(df_sub["creation_date"])) \
.withColumn("creation_day",dayofmonth(df_sub["creation_date"])) \
.show(5,False)

+--------+---------------------------------------------+-----+----------+------------------------------+-------------+--------------+------------+
|id      |tags                                         |score|view_count|creation_date                 |creation_year|creation_month|creation_day|
+--------+---------------------------------------------+-----+----------+------------------------------+-------------+--------------+------------+
|452884  |python|parsing|beautifulsoup                 |1    |1178      |2009-01-17 06:10:41.080000 UTC|2009         |1             |17          |
|92290   |visual-studio                                |1    |225       |2008-09-18 12:57:48.340000 UTC|2008         |9             |18          |
|6008898 |objective-c|c|printf|nslog                   |0    |217       |2011-05-15 14:11:23.250000 UTC|2011         |5             |15          |
|36737743|gradle                                       |1    |9         |2016-04-20 08:15:03.563000 UTC|2016         |

In [ ]:
df_sub.filter(df_sub.tags.like('python%